In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


#pdf = "Error_PDFS/BAQ_09_15_2022.pdf"
pdf = "Example_PDFS/equibaseFile.pdf"


In [3]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 1

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan

The PDF <_io.BufferedReader name='Error_PDFS/BAQ_09_15_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track :  BELMONT AT THE BIG A
date :  September 15, 2022
race_num :  2
criteria :  TO BE RUN OVER NATIONAL FENCES A HURDLE HANDICAP FOR FOUR-YEAR-OLDS AND UPWARD.
track_length :  (7)(e)(5) Lasix not permitted within 48 hours of post time. Closed Saturday, September 3, 2022 with 12 Nominations. Two And One Half Miles On The Hurdle
purse :  $150,000
weather :  Clear
track_type :  Firm
off_time :  1 43
start :  Good for all
track : Belmont At the Big A
date : September 15, 2022
race_num : 2
criteria : TO BE RUN OVER NATIONAL FENCES A HURDLE HANDICAP FOR FOUR-YEAR-OLDS AND UPWARD
track_length :  Two And One Half Miles On The Hurdle
purse : $150,000
weather : Clear
track_type : Firm
off_time : 1:43
start : Good for all
race_type : stakes
Looking for tables...
Found table top bound at 134.12199999999996
Found table bottom bound at 260.822
Looking for tables...
Found table top bound at 369.122
Couldn't read a table, moving on from 379.567 bottom_bound
Couldn't read a table, moving on from 369

Exception: Couldn't find top table

In [4]:
result_tables[1]

,Pgm,Horse Name,Start,1/4,3/8,Str,Fin,1/4_length_behind,3/8_length_behind,Str_length_behind,Fin_length_behind
0,4,Boisterously Irish,4.0,5,3,2,1,5,3 1/2,2,1 3/4
1,3,She Had a Secret,3.0,2,1,1,2,1 1/2,Head,2,1 3/4
2,2,Belle My Belle,5.0,4,4,3,3,4,3 3/4,2 1/2,3
3,6,Downtown Cutie,2.0,1,2,4,4,1 1/2,Head,5 1/2,11 1/2
4,1,Sweet and Ez,7.0,6,5,5,5,6,5 3/4,10,14 1/4
5,5,C'Mon Lets Switch,1.0,3,6,6,6,3 1/2,7 1/4,17,31
6,7,Legal Mischief,6.0,---,---,---,---,NaN,NaN,NaN,NaN


In [2]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    print(header)
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0].astype(object)
    bottom_table = result_tables[1].astype(object)
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 5}, {'page_num': 1, 'horse_count': 7}, {'page_num': 2, 'horse_count': 5}, {'page_num': 3, 'horse_count': 6}, {'page_num': 4, 'horse_count': 8}, {'page_num': 5, 'horse_count': 7}, {'page_num': 6, 'horse_count': 5}, {'page_num': 7, 'horse_count': 8}, {'page_num': 8, 'horse_count': 8}, {'page_num': 9, 'horse_count': 8}, {'page_num': 10, 'horse_count': 5}]
track :  MO <NMOUTH PARK
date :  August 27, 2022
race_num :  1
criteria :  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH HAVE BROKEN THEIR MAIDEN FOR A CLAIMING PRICE OR IN ANY STATE BRED RACE AND WHICH HAVE NEVER WON TWO RACES OR CLAIMING PRICE $50,000.
claiming_price :  $50,000.
track_length :  Five And One Half Furlongs On The Dirt
purse :  $45,000
weather :  Clear
track_type :  Fast
off_time :  12 16
start :  Good for all except 1
{'track': 'Monmouth Park', 'date': 'August 27, 2022', 'race_num': '1', 'criteria': '(PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH HAVE BROKEN THEIR MAIDEN FOR A CLAIMING PRICE 

The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table top bound at 404.8310000000001
Found table bottom bound at 479.4650000000001
    Last Raced  Pgm jockey_first_name jockey_last_name   PP Start 1/4 3/8 Str  \
0   2Jul22 MTH  4.0             Albin          Jimenez  4.0   3.0   2   2   2   
1   2Jul22 MTH  6.0             Angel        Rodriguez  5.0   2.0   3   3   1   
2  30Jun22 DEL  1.0              Jose           Ferrer  1.0   5.0   5   5   4   
3  22Jul22 MTH  2.0             Jorge      Vargas  Jr.  2.0   4.0   4   4   5   
4  30Jul22 MTH  3.0             Jomar           Torres  3.0   1.0   1   1   3   

  Fin  ... race_num                                           criteria  \
0   1  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
1   2  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
2   3  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
3   4  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR OLDS WHICH H...   
4   5  ...        1  (PLUS UP TO 40% NJB) FOR TWO YEAR

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

Found table bottom bound at 506.567
     Last Raced  Pgm jockey_first_name jockey_last_name   PP Start  1/4  3/8  \
0    2Jul22 MTH  4.0             Albin          Jimenez  4.0   3.0    2    2   
1    2Jul22 MTH  6.0             Angel        Rodriguez  5.0   2.0    3    3   
2   30Jun22 DEL  1.0              Jose           Ferrer  1.0   5.0    5    5   
3   22Jul22 MTH  2.0             Jorge      Vargas  Jr.  2.0   4.0    4    4   
4   30Jul22 MTH  3.0             Jomar           Torres  3.0   1.0    1    1   
5    5Aug22 MTH  5.0             Albin          Jimenez  4.0   7.0    6  NaN   
6   15Aug22 MTH  4.0          Madeline          Rowland  3.0   6.0    7  NaN   
7    5Aug22 MTH  6.0             Jairo           Rendon  5.0   3.0    2  NaN   
8   17Jul22 MTH  3.0             Angel        Rodriguez  2.0   4.0    5  NaN   
9   30Jul22 MTH  8.0            Carlos            Rojas  7.0   5.0    4  NaN   
10  30Jul22 MTH  7.0             Manny          Jimenez  6.0   2.0    3  NaN   
11  

Looking for tables...
Found table top bound at 143.12199999999996
Found table bottom bound at 234.42199999999988
Looking for tables...
Found table top bound at 439.997
Found table bottom bound at 542.567
     Last Raced  Pgm jockey_first_name  jockey_last_name   PP Start 1/4  3/8  \
0    2Jul22 MTH  4.0             Albin           Jimenez  4.0   3.0   2    2   
1    2Jul22 MTH  6.0             Angel         Rodriguez  5.0   2.0   3    3   
2   30Jun22 DEL  1.0              Jose            Ferrer  1.0   5.0   5    5   
3   22Jul22 MTH  2.0             Jorge       Vargas  Jr.  2.0   4.0   4    4   
4   30Jul22 MTH  3.0             Jomar            Torres  3.0   1.0   1    1   
..          ...  ...               ...               ...  ...   ...  ..  ...   
62   5Aug22 MTH  2.0             Manny           Jimenez  2.0   7.0   4  NaN   
63   9Jul22 MTH  7.0              Paco             Lopez  7.0   6.0   7  NaN   
64   1Aug22 MTH  5.0              Omar  Hernandez Moreno  5.0   4.0   2  NaN

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Example_PDFS/equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


 5 12
start :  Good for all
{'track': 'Monmouth Park', 'date': 'August 27, 2022', 'race_num': '11', 'criteria': '(PLUS UP TO 40% NJB) FOR FILLIES AND MARES THREE YEARS OLD AND UPWARD WHICH HAVE NEVER WON TWO RACES', 'claiming_price': '$30,000,', 'track_length': ' Six Furlongs On The Dirt', 'purse': '$30,000', 'weather': 'Clear', 'track_type': 'Fast', 'off_time': '5:12', 'start': 'Good for all', 'race_type': 'claiming'}
Looking for tables...
Found table top bound at 135.24599999999995
Found table bottom bound at 206.54599999999994
Looking for tables...
Found table top bound at 445.3310000000001
Found table bottom bound at 519.965
     Last Raced  Pgm jockey_first_name jockey_last_name   PP Start 1/4  3/8  \
0    2Jul22 MTH  4.0             Albin          Jimenez  4.0   3.0   2    2   
1    2Jul22 MTH  6.0             Angel        Rodriguez  5.0   2.0   3    3   
2   30Jun22 DEL  1.0              Jose           Ferrer  1.0   5.0   5    5   
3   22Jul22 MTH  2.0             Jorge      Var

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [3]:
pdf_df

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,Start,Str 1,Str,Fin,Odds,...,1/2_length_behind_RLP,claiming_price,3/8,3/8_length_behind,3/8_RLP,3/8_length_behind_RLP,3/4,3/4_length_behind,3/4_RLP,3/4_length_behind_RLP
0,3Sep22 AZD,6.0,Rodolfo,Arvizu,6.0,4.0,1,1,1,3.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3Sep22 AZD,7.0,Elias,Gutierrez,7.0,2.0,4,2,2,15.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3Sep22 AZD,3.0,David,Reyes-Frisby,3.0,5.0,3,3,3,1.70*,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15Aug22 AZD,1.0,Michael,Ybarra,1.0,1.0,2,4,4,7.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15Aug22 AZD,8.0,Martin,Osuna,8.0,3.0,5,5,5,9.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,29Aug22 AZD,4.0,Isaias,Enriquez,4.0,4.0,NaN,4,4,2.30,...,NaN,"$3,500",NaN,NaN,NaN,NaN,3,1 1/2,NaN,NaN
66,15Aug22 AZD,3.0,Devon,Ellis,3.0,5.0,NaN,6,5,19.10,...,NaN,"$3,500",NaN,NaN,NaN,NaN,6,Head,NaN,NaN
67,15Aug22 AZD,2.0,Kevin,Carbajal,2.0,2.0,NaN,5,6,22.30,...,NaN,"$3,500",NaN,NaN,NaN,NaN,4,1/2,NaN,NaN
68,29Aug22 AZD,5.0,Allyssa,Morales,5.0,6.0,NaN,7,7,7.70,...,NaN,"$3,500",NaN,NaN,NaN,NaN,7,3 1/2,NaN,NaN


In [6]:
pdf_df.to_csv("ARAP.csv")